### [PYTORCH C++ API](https://pytorch.org/cppdocs/)

#### [Installation](https://pytorch.org/cppdocs/installing.html)

In [ ]:
%%capture

!wget https://download.pytorch.org/libtorch/nightly/cpu/libtorch-shared-with-deps-latest.zip
!unzip libtorch-shared-with-deps-latest.zip

In [ ]:
!ls

libtorch  libtorch-shared-with-deps-latest.zip	sample_data


#### Hello world

In [ ]:
!mkdir -p example-app/build/

In [ ]:
%cd /content/example-app/

/content/example-app


In [ ]:
%%writefile example-app.cpp

#include <torch/torch.h>
#include <iostream>

int main() {
  torch::Tensor tensor = torch::rand({2, 3});
  std::cout << tensor << std::endl;
}

Writing example-app.cpp


In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(example-app)

find_package(Torch REQUIRED)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} ${TORCH_CXX_FLAGS}")

add_executable(example-app example-app.cpp)
target_link_libraries(example-app "${TORCH_LIBRARIES}")
set_property(TARGET example-app PROPERTY CXX_STANDARD 14)

Writing CMakeLists.txt


In [ ]:
%cd /content/example-app/build
!cmake -DCMAKE_PREFIX_PATH=/content/libtorch ..
!cmake --build . --config Release

/content/example-app/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Torch: /content/libtorch/lib/libtorch.so  
-- Configuring done
-- Generating done
-- Build files have been written to: /content/example-app/build
[ 50%] Building CXX object CMakeFiles/example-app.dir/example-app.cpp.o
[100%] Linking CXX executable example-app
[100%] Built target example-app


In [ ]:
!./example-app

 0.5236  0.4927  0.4325
 0.4201  0.5260  0.4001
[ CPUFloatType{2,3} ]


In [ ]:
%cd /content/

/content


#### ATen

[at:: namespace docs](https://pytorch.org/cppdocs/api/namespace_at.html#namespace-at)

ATen is fundamentally a tensor library, on top of which almost all other Python and C++ interfaces in PyTorch are built. It provides a core Tensor class, on which many hundreds of operations are defined. Most of these operations have both CPU and GPU implementations, to which the Tensor class will dynamically dispatch based on its type. A small example of using ATen could look as follows:

In [ ]:
#include <ATen/ATen.h>

at::Tensor a = at::ones({2, 2}, at::kInt);
at::Tensor b = at::randn({2, 2});
auto c = a + b.to(at::kInt);

#### Autograd

What we term autograd are the portions of PyTorch’s C++ API that augment the ATen Tensor class with capabilities concerning automatic differentiation. The autograd system records operations on tensors to form an autograd graph. Calling backwards() on a leaf variable in this graph performs reverse mode differentiation through the network of functions and tensors spanning the autograd graph, ultimately yielding gradients. The following example provides a taste of this interface:

In [ ]:
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

torch::Tensor a = torch::ones({2, 2}, torch::requires_grad());
torch::Tensor b = torch::randn({2, 2});
auto c = a + b;
c.backward(); // a.grad() will now hold the gradient of c w.r.t. a.

**The at::Tensor class in ATen is not differentiable by default. To add the differentiability of tensors the autograd API provides, you must use tensor factory functions from the torch:: namespace instead of the at:: namespace. For example, while a tensor created with at::ones will not be differentiable, a tensor created with torch::ones will be.**

#### C++ Frontend

[Docs](https://pytorch.org/cppdocs/frontend.html)

The PyTorch C++ frontend provides a high level, pure C++ modeling interface for neural network and general ML(Machine Learning) research and production use cases, largely following the Python API in design and provided functionality.

**Note:** Unless you have a particular reason to constrain yourself exclusively to ATen or the Autograd API, the C++ frontend is the recommended entry point to the PyTorch C++ ecosystem.

#### Examples for autograd and c++ frontend

https://github.com/pytorch/examples/tree/main/cpp/autograd

https://pytorch.org/docs/stable/notes/extending.html#example

#### TorchScript

TorchScript is an intermediate representation of a PyTorch model that can then be run in a high-performance environment such as C++.

https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html

https://pytorch.org/tutorials/advanced/cpp_export.html

https://pytorch.org/tutorials/advanced/torch_script_custom_ops.html

https://github.com/pytorch/extension-script/tree/master/example_app

#### C++ extensions

C++ Extensions offer a simple yet powerful way of accessing all of the above interfaces for the purpose of extending regular Python use-cases of PyTorch. C++ extensions are most commonly used to implement custom operators in C++ or CUDA to accelerate research in vanilla PyTorch setups. The C++ extension API does not add any new functionality to the PyTorch C++ API. Instead, it provides integration with Python setuptools as well as JIT compilation mechanisms that allow access to ATen, the autograd and other C++ APIs from Python

https://pytorch.org/docs/stable/notes/extending.html#example - Python custom operators
https://github.com/pytorch/examples/blob/main/cpp/autograd/autograd.cpp - C++ custom operators

In [ ]:
!mkdir -p custom_op/build/

In [ ]:
%cd /content/custom_op

/content/custom_op


In [ ]:
%%writefile mulconstant.cpp

#include <torch/torch.h>
#include <iostream>

using namespace torch::autograd;

class MulConstant : public Function<MulConstant> {
 public:
  static torch::Tensor forward(AutogradContext *ctx, torch::Tensor tensor, double constant) {
    // ctx is a context object that can be used to stash information
    // for backward computation
    ctx->saved_data["constant"] = constant;
    return tensor * constant;
  }

  static tensor_list backward(AutogradContext *ctx, tensor_list grad_outputs) {
    // We return as many input gradients as there were arguments.
    // Gradients of non-tensor arguments to forward must be `torch::Tensor()`.
    return {grad_outputs[0] * ctx->saved_data["constant"].toDouble(), torch::Tensor()};
  }
};

void custom_autograd_function_example() {
  std::cout << "====== Running \"Using custom autograd function in C++\" ======" << std::endl;
  {
    auto x = torch::randn({2}).requires_grad_();
    auto y = MulConstant::apply(x, 5.5);
    y.sum().backward();

    std::cout << "Градиент:" << x.grad() << std::endl;
  }
}

int main() {
  custom_autograd_function_example();
}

Writing mulconstant.cpp


In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 2.8)

project(custom_op)
set(CMAKE_CXX_STANDARD 14)

find_package(Torch REQUIRED)

add_executable(${PROJECT_NAME} "mulconstant.cpp")
target_link_libraries(${PROJECT_NAME} "${TORCH_LIBRARIES}")

Writing CMakeLists.txt


In [ ]:
%cd build

/content/custom_op/build


In [ ]:
!cmake -DCMAKE_PREFIX_PATH=/content/libtorch ..
!make

CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Torch: /content/libtorch/lib/libtorch.so  
-- Configuring done
-- Generating done
-- Build files have been written to: /content/custom_op/build
[ 50%] Building CXX object CMakeFi

In [ ]:
!./custom_op

====== Running "Using custom autograd function in C++" ======
Градиент: 5.5000
 5.5000
[ CPUFloatType{2} ]


In [ ]:
%cd /content/

/content


### Advanced example

This recurrent unit is similar to an LSTM, but differs in that it lacks a forget gate and uses an Exponential Linear Unit (ELU) as its internal activation function. Because this unit never forgets, we’ll call it LLTM, or Long-Long-Term-Memory unit.

In [ ]:
import torch

X = torch.randn(batch_size, input_features)
h = torch.randn(batch_size, state_size)
C = torch.randn(batch_size, state_size)

rnn = LLTM(input_features, state_size)

new_h, new_C = rnn(X, (h, C))

https://pytorch.org/tutorials/advanced/cpp_extension.html?highlight=c
https://github.com/pytorch/tutorials/blob/master/advanced_source/cpp_extension.rst?ysclid=ldexhqvmhe807255845  
https://github.com/pytorch/extension-cpp

Домашнее задание:

Реализовать выбранный оператор на Python и с++. Любым их рассмотренных способов провести запуск Python кода в c++ Frontend и с++ кода в .py